In [1]:
!pip install psycopg2-binary
!pip install s3fs
!pip install --upgrade boto3 botocore
!pip install minio

  Using cached botocore-1.29.76-py3-none-any.whl (10.4 MB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.29.132
    Uninstalling botocore-1.29.132:
      Successfully uninstalled botocore-1.29.132
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
boto3 1.26.132 requires botocore<1.30.0,>=1.29.132, but you have botocore 1.29.76 which is incompatible.
  Using cached botocore-1.29.132-py3-none-any.whl (10.7 MB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.29.76
    Uninstalling botocore-1.29.76:
      Successfully uninstalled botocore-1.29.76
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.5.0 requires botocore<1.29.77,>=1.29.76, but you have botocore 1.29.132 which

In [2]:
import pandas as pd
import io
import os
import boto3
import pyspark
import s3fs
import pyarrow as pa
import pyarrow.parquet as pq
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_json, struct
from pyspark.sql.types import StructType,StructField, StringType, IntegerType 
from pyspark.sql.types import ArrayType, DoubleType, BooleanType
from pyspark.sql.functions import col,array_contains
from minio import Minio
from minio.error import S3Error
from sqlalchemy import create_engine
import psycopg2

In [3]:
# Criando uma sessão com o Spark que existe localmente(atualmente configurado junto com o JupyterLab)
spark = SparkSession.builder \
      .master("local[1]") \
      .appName("AwariAula08") \
      .getOrCreate() 

In [4]:
spark.stop()

conf = pyspark.SparkConf()

# Criando uma sessão com o Spark que existe localmente(atualmente configurado junto com o JupyterLab)
conf.setMaster("local[1]") 
conf.set("spark.driver.host", "awari-jupyterlab") \
    .set("spark.sql.sources.commitProtocolClass", "org.apache.spark.sql.execution.datasources.SQLHadoopMapReduceCommitProtocol") \
    .set("parquet.enable.summary-metadata", "false") \
    .set("mapreduce.fileoutputcommitter.marksuccessfuljobs", "false") \
    .set("spark.driver.port", "20020") \
    .set("spark.hadoop.fs.s3a.endpoint", 'awari-nginx:9000') \
    .set("spark.hadoop.fs.s3a.endpoint.region", 'sa-east-1') \
    .set("spark.hadoop.fs.s3a.access.key", '4Xf3Q7unyUtmTVFH') \
    .set("spark.hadoop.fs.s3a.secret.key", 'q0b3SiIc28hy0750XN3uL1u5G5CzW0hR') \
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .set("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .set("spark.hadoop.com.amazonaws.services.s3.enableV2", "true") \
    .set("spark.hadoop.fs.s3a.committer.staging.conflict-mode", "replace") \
    .set("spark.hadoop.fs.s3a.fast.upload", True) \
    .set("spark.hadoop.fs.s3a.path.style.access", True) \
    .set("spark.hadoop.fs.s3a.committer.name", "directory") \
    .set("spark.hadoop.fs.s3a.committer.staging.tmp.path", "/tmp/staging")

conf.setAppName('AwariAula08-S33')
sc = pyspark.SparkContext(conf=conf)

spark = SparkSession(sc)

In [5]:
client = boto3.client('s3', 
    endpoint_url='http://awari-minio-nginx:9000',
    aws_access_key_id='4Xf3Q7unyUtmTVFH',
    aws_secret_access_key='q0b3SiIc28hy0750XN3uL1u5G5CzW0hR',
    aws_session_token=None,
    config=boto3.session.Config(signature_version='s3v4'),
    verify=False,
    region_name='sa-east-1'
)

In [6]:
# Importar CSVs como dataframes do Spark
df_estados = spark.read.option("header", True).csv("exercicios/municipios-estados/csv/estados.csv")
df_municipios = spark.read.option("header", True).csv("exercicios/municipios-estados/csv/municipios.csv")

In [7]:
df_estados.take(5)

[Row(codigo_uf='11', uf='RO', nome='Rondônia', latitude='-10.83', longitude='-63.34', regiao='Norte'),
 Row(codigo_uf='12', uf='AC', nome='Acre', latitude='-8.77', longitude='-70.55', regiao='Norte'),
 Row(codigo_uf='13', uf='AM', nome='Amazonas', latitude='-3.47', longitude='-65.1', regiao='Norte'),
 Row(codigo_uf='14', uf='RR', nome='Roraima', latitude='1.99', longitude='-61.33', regiao='Norte'),
 Row(codigo_uf='15', uf='PA', nome='Pará', latitude='-3.79', longitude='-52.48', regiao='Norte')]

In [8]:
df_municipios.take(5)

[Row(codigo_ibge='5200050', nome='Abadia de Goiás', latitude='-16.7573', longitude='-49.4412', capital='0', codigo_uf='52', siafi_id='1050', ddd='62', fuso_horario='America/Sao_Paulo'),
 Row(codigo_ibge='3100104', nome='Abadia dos Dourados', latitude='-18.4831', longitude='-47.3916', capital='0', codigo_uf='31', siafi_id='4001', ddd='34', fuso_horario='America/Sao_Paulo'),
 Row(codigo_ibge='5200100', nome='Abadiânia', latitude='-16.197', longitude='-48.7057', capital='0', codigo_uf='52', siafi_id='9201', ddd='62', fuso_horario='America/Sao_Paulo'),
 Row(codigo_ibge='3100203', nome='Abaeté', latitude='-19.1551', longitude='-45.4444', capital='0', codigo_uf='31', siafi_id='4003', ddd='37', fuso_horario='America/Sao_Paulo'),
 Row(codigo_ibge='1500107', nome='Abaetetuba', latitude='-1.72183', longitude='-48.8788', capital='0', codigo_uf='15', siafi_id='0401', ddd='91', fuso_horario='America/Sao_Paulo')]

In [9]:
# Transformando nosso DataFrame para PANDAS
df_pandas_estados = df_estados.toPandas()
df_pandas_municipios = df_municipios.toPandas()

In [10]:
df_pandas_estados.head(5)

,codigo_uf,uf,nome,latitude,longitude,regiao
0,11,RO,Rondônia,-10.83,-63.34,Norte
1,12,AC,Acre,-8.77,-70.55,Norte
2,13,AM,Amazonas,-3.47,-65.1,Norte
3,14,RR,Roraima,1.99,-61.33,Norte
4,15,PA,Pará,-3.79,-52.48,Norte


In [11]:
df_pandas_municipios.head(5)

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,siafi_id,ddd,fuso_horario
0,5200050,Abadia de Goiás,-16.7573,-49.4412,0,52,1050,62,America/Sao_Paulo
1,3100104,Abadia dos Dourados,-18.4831,-47.3916,0,31,4001,34,America/Sao_Paulo
2,5200100,Abadiânia,-16.197,-48.7057,0,52,9201,62,America/Sao_Paulo
3,3100203,Abaeté,-19.1551,-45.4444,0,31,4003,37,America/Sao_Paulo
4,1500107,Abaetetuba,-1.72183,-48.8788,0,15,0401,91,America/Sao_Paulo


In [12]:
csv_path = 'exercicios/municipios-estados/csv/'
total_path = 'exercicios/municipios-estados/tarefa-08/'

In [13]:
# Importar CSVs e salvar como arquivos JSON
csv_files = os.listdir(csv_path)
csv_files = [f for f in csv_files if f.endswith('.csv')]
for csv_file in csv_files:
    df = pd.read_csv(os.path.join(csv_path, csv_file))
    json_file = csv_file.replace('.csv', '.json')
    df.to_json(os.path.join(total_path, json_file), orient='records')

In [14]:
# Importar JSONs e salvar como arquivos CSV
json_files = os.listdir(total_path)
json_files = [f for f in json_files if f.endswith('.json')]
for json_file in json_files:
    df = pd.read_json(os.path.join(total_path, json_file), orient='records')
    csv_file = json_file.replace('.json', '.csv')
    df.to_csv(os.path.join(total_path, csv_file), index=False)

In [15]:
# Importar CSVs e salvar como arquivos Parquet
csv_files = os.listdir(csv_path)
csv_files = [f for f in csv_files if f.endswith('.csv')]
for csv_file in csv_files:
    df = pd.read_csv(os.path.join(csv_path, csv_file))
    parquet_file = csv_file.replace('.csv', '.parquet')
    table = pa.Table.from_pandas(df)
    pq.write_table(table, os.path.join(total_path, parquet_file))

In [16]:
# lista todos os arquivos na pasta de origem
for root, dirs, files in os.walk(total_path):
    for file in files:
        # obtém o caminho completo do arquivo
        file_path = os.path.join(root, file)
        
        # define o nome do objeto no bucket (pode ser igual ao nome do arquivo)
        object_name = file
        
        # faz upload do arquivo para o bucket
        with open(file_path, "rb") as f:
            client.upload_fileobj(f, "aula-08", object_name)

In [17]:
# Salvar dataframes como arquivos JSON no MinIO
#df_estados_local.write.format("json").save("s3a://aula-08/municipios-estados/csv/estados.json")
#df_municipios_local.write.format("json").save("s3a://aula-08/municipios-estados/csv/municipios.json")

# Importar JSONs do MinIO como dataframes do Spark
#df_estados_json = spark.read.option("header", True).json("s3a://aula-08/municipios-estados/csv/estados.json")
#df_municipios_json = spark.read.option("header", True).json("s3a://aula-08/municipios-estados/csv/municipios.json")

# Salvar dataframes como arquivos CSV no MinIO
#df_estados_json.write.format("csv").option("header", True).save("s3a://aula-08/municipios-estados/csv/estados.csv")
#df_municipios_json.write.format("csv").option("header", True).save("s3a://aula-08/municipios-estados/csv/municipios.csv")

# Importar CSVs do MinIO como dataframes do Spark
#df_estados_csv = spark.read.option("header", True).csv("s3a://aula-08/municipios-estados/csv/estados.csv")
#df_municipios_csv = spark.read.option("header", True).csv("s3a://aula-08/municipios-estados/csv/municipios.csv")

# Salvar dataframes como arquivos .parquet no MinIO
#df_estados_csv.write.save("s3a://aula-08/municipios-estados/parquet/estados.parquet")
#df_municipios_csv.write.save("s3a://aula-08/municipios-estados/parquet/municipios.parquet")

In [20]:
# Configurar as credenciais do banco de dados
db_config = {
    'user': 'postgres',
    'password': 'postgres',
    'host': '172.18.0.6',
    'port': 5432,
    'database': 'awari_imdb'
}

# Conectar ao banco de dados
conn = psycopg2.connect(**db_config)

# Ler o arquivo CSV para um DataFrame pandas
df_estados = pd.read_csv('exercicios/municipios-estados/csv/estados.csv')
df_municipios = pd.read_csv('exercicios/municipios-estados/csv/municipios.csv')

# Inserir os dados do DataFrame no PostgreSQL
with conn.cursor() as cur:
    # Inserir os dados do DataFrame de estados
    cur.execute("""
        CREATE TABLE estados (
            codigo_uf INTEGER PRIMARY KEY,
            uf TEXT,
            nome TEXT
        );
    """)
    for row in df_estados.itertuples(index=False):
        cur.execute("""
            INSERT INTO estados (codigo_uf, uf, nome) VALUES (%s, %s, %s);
        """, (row.codigo_uf, row.uf, row.nome))
        
    # Inserir os dados do DataFrame de municipios
    cur.execute("""
        CREATE TABLE municipios (
            codigo_ibge INTEGER PRIMARY KEY,
            nome TEXT,
            codigo_uf INTEGER,
            FOREIGN KEY (codigo_uf) REFERENCES estados(codigo_uf)
        );
    """)
    for row in df_municipios.itertuples(index=False):
        cur.execute("""
            INSERT INTO municipios (codigo_ibge, nome, codigo_uf) VALUES (%s, %s, %s);
        """, (row.codigo_ibge, row.nome, row.codigo_uf))
        
    # Commit as transações
    conn.commit()

# Fechar a conexão com o banco de dados
conn.close()

DuplicateTable: relation "estados" already exists


In [22]:
#df_estados_csv = spark.read.option("header", True).csv("s3a://aula-08/municipios-estados/csv/estados.csv")
#print(df_estados.show(), df_estados.printSchema())

In [23]:
#df_municipios_csv = spark.read.option("header", True).csv("s3a://aula-08/municipios-estados/csv/municipios.csv")
#print(df_municipios.show(), df_municipios.printSchema())

In [24]:
# df_estado_json = df_estado.select(to_json(struct(df_estado.columns)).alias("json"))
# df_municipio_json = df_municipio.select(to_json(struct(df_municipio.columns)).alias("json"))